In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Join').getOrCreate()

25/03/15 03:53:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
spark.sparkContext.applicationId

'application_1742006069642_0005'

In [5]:
spark.conf.set('spark.sql.adaptive.enabled','false')

In [6]:
bucket = 'gs://dataproc-staging-us-central1-458263062208-tw36mmqt'
file_path = '/notebooks/jupyter/Data/'

In [7]:
customers_df = spark.read.csv(bucket+file_path+'customers.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(bucket+file_path+'orders.csv',header=True,inferSchema=True)

In [8]:
customers_df.show(2),orders_df.show(2)

+-----------+----------+-------+-----------+-------+-------------------+---------+
|customer_id|      name|   city|      state|country|  registration_date|is_active|
+-----------+----------+-------+-----------+-------+-------------------+---------+
|          0|Customer_0| Mumbai|  Telangana|  India|2023-03-21 00:00:00|     true|
|          1|Customer_1|Chennai|West Bengal|  India|2023-05-27 00:00:00|    false|
+-----------+----------+-------+-----------+-------+-------------------+---------+
only showing top 2 rows

+--------+-----------+-------------------+-----------------+-------+
|order_id|customer_id|         order_date|     total_amount| status|
+--------+-----------+-------------------+-----------------+-------+
|       0|    3194509|2024-04-12 00:00:00|772.3507972244216|Shipped|
|       1|    8003925|2024-05-06 00:00:00| 404.843827081314|Shipped|
+--------+-----------+-------------------+-----------------+-------+
only showing top 2 rows



(None, None)

In [9]:
!hadoop fs -ls -h /data2/

Found 1 items
-rw-r--r--   2 tech_mayankagg hadoop      1.0 M 2025-03-01 03:34 /data2/customers.csv


In [10]:
customers_small_df = spark.read.csv('/data2/customers.csv',header=True,inferSchema=True)

In [11]:
customers_small_df.show(2)

+-----------+----------+---------+-----------+-------+-------------------+---------+
|customer_id|      name|     city|      state|country|  registration_date|is_active|
+-----------+----------+---------+-----------+-------+-------------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|2023-06-29 00:00:00|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|2023-12-07 00:00:00|     true|
+-----------+----------+---------+-----------+-------+-------------------+---------+
only showing top 2 rows



In [12]:
joined_df = orders_df.join(customers_small_df,on='customer_id',how='inner')

We seem to be working with identical names which is always not the case.

In [13]:
# Don't do show() as that might take less partition

joined_df.write.format('noop').mode('overwrite').save()

In [14]:
customers_small_df.count()

17653

In [ ]:
spark.conf.set('spark.sql.adaptive.enabled','false')


In [16]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold',-1)

In [17]:
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'-1'

In [18]:
orders_df.join(customers_small_df,on='customer_id',how='inner').write.format('noop').mode('overwrite').save()

Hi Mayank , I didn't receive any mail today so thought there was no class today! later on came to know class was there. Could you let me know what are we covering today?

In [24]:
orders_df.selectExpr('customer_id', 'customer_id % 200 as cust_key').show()
customers_small_df.selectExpr('customer_id', 'customer_id % 200 as cust_key').show()

+-----------+--------+
|customer_id|cust_key|
+-----------+--------+
|    3194509|     109|
|    8003925|     125|
|    1946602|       2|
|    8511049|      49|
|    5013836|      36|
|    5352874|      74|
|    3676047|      47|
|     663456|      56|
|    5471172|     172|
|    8298598|     198|
|    2217320|     120|
|    4796277|      77|
|     766907|     107|
|    3924592|     192|
|    7014407|       7|
|    4008694|      94|
|    8248295|      95|
|     372605|       5|
|    7289017|      17|
|    6406178|     178|
+-----------+--------+
only showing top 20 rows

+-----------+--------+
|customer_id|cust_key|
+-----------+--------+
|          0|       0|
|          1|       1|
|          2|       2|
|          3|       3|
|          4|       4|
|          5|       5|
|          6|       6|
|          7|       7|
|          8|       8|
|          9|       9|
|         10|      10|
|         11|      11|
|         12|      12|
|         13|      13|
|         14|      14|
|       

In [26]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold','50m')

In [27]:
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'50m'

In [28]:
orders_df.join(customers_small_df,on='customer_id',how='inner').write.format('noop').mode('overwrite').save()

# Skew Join

In [29]:
orders_df.count(),customers_df.count()

(8767358, 8767358)

In [34]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: timestamp (nullable = true)
 |-- is_active: boolean (nullable = true)



In [35]:
import pyspark.sql.functions as F

In [33]:
customers_df.groupBy('city').count().show()

+---------+-------+
|     city|  count|
+---------+-------+
|Bangalore|1094195|
|  Chennai|1095052|
|   Mumbai|1095815|
|Ahmedabad|1097162|
|  Kolkata|1096777|
|     Pune|1095748|
|    Delhi|1096183|
|Hyderabad|1096426|
+---------+-------+



In [38]:
skew_factor = 1000

In [39]:
mumbai_df = customers_df.filter(F.col('city')=='Mumbai')

In [41]:
mumbai_df.count()

1095815

In [42]:
# Generate multiple copies of Mumbai's data using an array expansion trick
replicated_mumbai_df = mumbai_df.withColumn("repeat", F.explode(F.array([F.lit(i) for i in range(skew_factor)])))
replicated_mumbai_df.count()

1095815000

In [44]:


# Adjust customer_id to avoid duplicates
replicated_mumbai_df = replicated_mumbai_df.withColumn("customer_id",(F.col("customer_id") + F.monotonically_increasing_id()) % (10**8)).drop("repeat")
# Combine the skewed Mumbai data back into the main dataset
customers_big_customers_dfskewed_df = customers_df.union(replicated_mumbai_df)
# Verify the new distribution
customers_big_customers_dfskewed_df.groupBy("city").count().show()

+---------+----------+
|     city|     count|
+---------+----------+
|Bangalore|   1094195|
|  Chennai|   1095052|
|   Mumbai|2192725815|
|Ahmedabad|   1097162|
|  Kolkata|   1096777|
|     Pune|   1095748|
|    Delhi|   1096183|
|Hyderabad|   1096426|
+---------+----------+



In [45]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold',-1)
spark.conf.set('spark.sql.adaptive.enabled','false')

In [46]:
city_data = [
    ("Mumbai", "High Population"),
    ("Bangalore", "IT Hub"),
    ("Hyderabad", "Tech City"),
    ("Delhi", "Capital"),
    ("Chennai", "Coastal City"),
    ("Pune", "Education Hub"),
    ("Ahmedabad", "Business Center"),
    ("Kolkata", "Cultural City"),
]

city_columns = ["city", "description"]
city_df = spark.createDataFrame(city_data, city_columns)


In [47]:
city_df.count()

8

In [48]:
joined_df = customers_big_customers_dfskewed_df.join(city_df,'city','inner')

In [49]:
joined_df.write.format('noop').mode('overwrite').save()

25/03/15 05:59:27 ERROR OverwriteByExpressionExec: Data source write support org.apache.spark.sql.execution.datasources.noop.NoopBatchWrite$@ccc9d74 is aborting.
25/03/15 05:59:27 ERROR OverwriteByExpressionExec: Data source write support org.apache.spark.sql.execution.datasources.noop.NoopBatchWrite$@ccc9d74 aborted.


Py4JJavaError: An error occurred while calling o1235.save.
: org.apache.spark.SparkException: Writing job aborted
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobAbortedError(QueryExecutionErrors.scala:766)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:409)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:353)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExec.writeWithV2(WriteToDataSourceV2Exec.scala:262)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run(WriteToDataSourceV2Exec.scala:332)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run$(WriteToDataSourceV2Exec.scala:331)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExec.run(WriteToDataSourceV2Exec.scala:262)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute

explain the term skewness with example

Our executor has storage memory as well, right? Why can’t the job get successful by spilling data to disk?